In [13]:
import numpy as np
import scipy.misc
import PIL.Image
import scipy.io
import os
import cv2
import argparse

def parse_args():
    parser = argparse.ArgumentParser(description='batch proccesing: photos->edges')
    parser.add_argument('--caffe_root', dest='caffe_root', help='caffe root', default='/opt/caffe/', type=str)
    parser.add_argument('--caffemodel', dest='caffemodel', help='caffemodel', default='/opt/caffe/examples/hed/hed_pretrained_bsds.caffemodel', type=str)
    parser.add_argument('--prototxt', dest='prototxt', help='caffe prototxt file', default='/opt/caffe/examples/hed/deploy.prototxt', type=str)
    parser.add_argument('--images_dir', dest='images_dir', help='directory to store input photos', type=str)
    parser.add_argument('--hed_mat_dir', dest='hed_mat_dir', help='directory to store output hed edges in mat file',  type=str)
    parser.add_argument('--border', dest='border', help='padding border', type=int, default=128)
    args = parser.parse_args()
    return args

In [18]:
args_ = parse_args()
for arg in vars(args_):
    print(getattr(args_, arg))
    print('[%s] =' % arg, getattr(args, arg))
import caffe
import scipy.io as sio

usage: ipykernel_launcher.py [-h] [--caffe_root CAFFE_ROOT]
                             [--caffemodel CAFFEMODEL] [--prototxt PROTOTXT]
                             [--images_dir IMAGES_DIR]
                             [--hed_mat_dir HED_MAT_DIR] [--border BORDER]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-722bb6fe-8983-4fc2-ad1e-89f98ba6e5bf.json


SystemExit: 2

In [17]:
%tb

SystemExit: 2

In [ ]:
 if not os.path.exists(args.hed_mat_dir):
    print('create output directory %s' % args.hed_mat_dir)
    os.makedirs(args.hed_mat_dir)

imgList = os.listdir(args.images_dir)
nImgs = len(imgList)
print('#images = %d' % nImgs)

# load net
net = caffe.Net(args.prototxt, args.caffemodel, caffe.TEST)
# pad border
border = args.border    

for i in range(nImgs):
    if i % 500 == 0:
        print('processing image %d/%d' % (i, nImgs))
    im = Image.open(os.path.join(args.images_dir, imgList[i]))

    in_ = np.array(im, dtype=np.float32)
    in_ = np.pad(in_,((border, border),(border,border),(0,0)),'reflect')

    in_ = in_[:,:,::-1]
    in_ -= np.array((104.00698793,116.66876762,122.67891434))
    in_ = in_.transpose((2, 0, 1))
    # remove the following two lines if testing with cpu

    # shape for input (data blob is N x C x H x W), set data
    net.blobs['data'].reshape(1, *in_.shape)
    net.blobs['data'].data[...] = in_
    # run net and take argmax for prediction
    net.forward()
    fuse = net.blobs['sigmoid-fuse'].data[0][0, :, :]
    # get rid of the border
    fuse = fuse[border:-border, border:-border]
    # save hed file to the disk
    name, ext = os.path.splitext(imgList[i])
sio.savemat(os.path.join(args.hed_mat_dir, name + '.mat'), {'predict':fuse})